In [9]:
import pandas as pd
import random
from datetime import datetime, timedelta


supermarkets = ['Naivas', 'Quickmart', 'Carrefour', 'Tuskys', 'Chandarana', 'Uchumi']
mpesa_agents = ['MPesa Agent 25401', 'MPesa Agent 25402', 'MPesa Agent 25403']

data = []
start_date = datetime(2025, 4, 1)
transaction_types = ['Buy Goods', 'Withdrawal', 'Deposit', 'Send Money', 'Paybill']

for i in range(50):
    date = start_date + timedelta(days=random.randint(0, 59), hours=random.randint(0, 23), minutes=random.randint(0, 59))
    data.append({
        'transaction_id': f'MP{random.randint(100000, 999999)}',
        'customer_phone': f'+2547{random.randint(10_000_000, 99_999_999)}',
        'supermarket': random.choice(supermarkets),
        'agent': random.choice(mpesa_agents),
        'transaction_type': random.choice(transaction_types),
        'amount': random.randint(100, 10000),
        'date': date.strftime('%Y-%m-%d'),
        'last_updated': date.strftime('%Y-%m-%d %H:%M:%S')
    })

df = pd.DataFrame(data)
df.to_csv('custom_data.csv', index=False)
df.head()

,transaction_id,customer_phone,supermarket,agent,transaction_type,amount,date,last_updated
0,MP293758,+254727515908,Tuskys,MPesa Agent 25401,Send Money,3944,2025-04-13,2025-04-13 08:53:00
1,MP712864,+254786882480,Chandarana,MPesa Agent 25401,Buy Goods,8359,2025-04-30,2025-04-30 06:03:00
2,MP773542,+254768442195,Uchumi,MPesa Agent 25402,Send Money,1821,2025-05-22,2025-05-22 12:07:00
3,MP173997,+254774408743,Carrefour,MPesa Agent 25403,Deposit,5523,2025-04-21,2025-04-21 15:37:00
4,MP238917,+254778761760,Tuskys,MPesa Agent 25403,Deposit,2440,2025-05-17,2025-05-17 07:35:00


In [10]:
df_full = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
print(f"Extracted {len(df_full)} rows fully.")
print(f"Columns: {list(df_full.columns)}")
df_full.head()

Extracted 50 rows fully.
Columns: ['transaction_id', 'customer_phone', 'supermarket', 'agent', 'transaction_type', 'amount', 'date', 'last_updated']


,transaction_id,customer_phone,supermarket,agent,transaction_type,amount,date,last_updated
0,MP293758,254727515908,Tuskys,MPesa Agent 25401,Send Money,3944,2025-04-13,2025-04-13 08:53:00
1,MP712864,254786882480,Chandarana,MPesa Agent 25401,Buy Goods,8359,2025-04-30,2025-04-30 06:03:00
2,MP773542,254768442195,Uchumi,MPesa Agent 25402,Send Money,1821,2025-05-22,2025-05-22 12:07:00
3,MP173997,254774408743,Carrefour,MPesa Agent 25403,Deposit,5523,2025-04-21,2025-04-21 15:37:00
4,MP238917,254778761760,Tuskys,MPesa Agent 25403,Deposit,2440,2025-05-17,2025-05-17 07:35:00


In [11]:
with open("last_extraction.txt", "w") as f:
    f.write("2025-04-15 08:00:00")

In [12]:
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()
df = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
last_extraction_time = pd.to_datetime(last_extraction)
df_incremental = df[df['last_updated'] > last_extraction_time]
print(f"Extracted {len(df_incremental)} rows incrementally since {last_extraction}.")
df_incremental.head()

Extracted 38 rows incrementally since 2025-04-15 08:00:00.


,transaction_id,customer_phone,supermarket,agent,transaction_type,amount,date,last_updated
1,MP712864,254786882480,Chandarana,MPesa Agent 25401,Buy Goods,8359,2025-04-30,2025-04-30 06:03:00
2,MP773542,254768442195,Uchumi,MPesa Agent 25402,Send Money,1821,2025-05-22,2025-05-22 12:07:00
3,MP173997,254774408743,Carrefour,MPesa Agent 25403,Deposit,5523,2025-04-21,2025-04-21 15:37:00
4,MP238917,254778761760,Tuskys,MPesa Agent 25403,Deposit,2440,2025-05-17,2025-05-17 07:35:00
5,MP224240,254786614056,Chandarana,MPesa Agent 25401,Buy Goods,6725,2025-05-25,2025-05-25 06:39:00


In [13]:
if not df.empty:
    new_checkpoint = df['last_updated'].max()
    with open("last_extraction.txt", "w") as f:
        f.write(str(new_checkpoint))
    print(f"Updated last_extraction.txt to {new_checkpoint}")

Updated last_extraction.txt to 2025-05-29 16:27:00
